In [11]:
#! -*- coding: utf-8 -*-
import glob
import numpy as np
import shutil

from keras.preprocessing.image import load_img, img_to_array, array_to_img
from keras.preprocessing.image import random_rotation, random_shift, random_zoom
from keras.layers.convolutional import Conv2D
from keras.layers.pooling import MaxPooling2D
from keras.layers.core import Activation
from keras.layers.core import Dense
from keras.layers.core import Dropout
from keras.layers.core import Flatten
from keras.models import Sequential
from keras.models import model_from_json
from keras.callbacks import LearningRateScheduler
from keras.callbacks import ModelCheckpoint
from keras.optimizers import Adam
from keras.utils import np_utils

import matplotlib.pyplot as plt
%matplotlib inline

FileNames = ["gami.npy", "kaoru.npy", "soeda.npy","reiya.npy","mattyan.npy","kasuya.npy"]
ClassNames = ["gomi","kaoru", "soeda", "reiya","mattyan","kasuya"]
hw = {"height":32, "width":32}        # リストではなく辞書型 中かっこで囲む


################################
###### 画像データの前処理 ######
################################
def PreProcess(dirname, filename, var_amount=3):
    num = 0
    arrlist = []
    files = glob.glob(dirname + "/*.jpg")

    for imgfile in files:
        img = load_img(imgfile, target_size=(hw["height"], hw["width"]))    # 画像ファイルの読み込み
        array = img_to_array(img) / 255                                     # 画像ファイルのnumpy化
        arrlist.append(array)                 # numpy型データをリストに追加
        for i in range(var_amount-1):
            arr2 = array
            arr2 = random_rotation(arr2, rg=360)
            arrlist.append(arr2)              # numpy型データをリストに追加
        num += 1

    nplist = np.array(arrlist)
    np.save(filename, nplist)
    print(">> " + dirname + "から" + str(num) + "個のファイル読み込み成功")


################################
######### モデルの構築 #########
################################
def BuildCNN(ipshape=(32, 32, 3), num_classes=3):
    model = Sequential()

    model.add(Conv2D(24, 3, padding='same', input_shape=ipshape))
    model.add(Activation('relu'))

    model.add(Conv2D(48, 3))
    model.add(Activation('relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Dropout(0.5))

    model.add(Conv2D(96, 3, padding='same'))
    model.add(Activation('relu'))

    model.add(Conv2D(96, 3))
    model.add(Activation('relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Dropout(0.5))

    model.add(Flatten())
    model.add(Dense(128))
    model.add(Activation('relu'))
    model.add(Dropout(0.5))

    model.add(Dense(num_classes))
    model.add(Activation('softmax'))

    adam = Adam(lr=0.001, beta_1=0.9, beta_2=0.999, epsilon=1e-08)
    model.compile(loss='categorical_crossentropy',
                  optimizer=adam,
                  metrics=['accuracy'])
    return model


################################
############# 学習 #############
################################
def Learning(tsnum=30, nb_epoch=50, batch_size=8, learn_schedule=0.9):
    X_TRAIN_list = []; Y_TRAIN_list = []; X_TEST_list = []; Y_TEST_list = [];
    target = 0
    for filename in FileNames :
        data = np.load(filename)          # 画像のnumpyデータを読み込み
        trnum = data.shape[0] - tsnum
        X_TRAIN_list += [data[i] for i in range(trnum)]          # 画像データ
        Y_TRAIN_list += [target] * trnum                         # 分類番号
        X_TEST_list  += [data[i] for i in range(trnum, trnum+tsnum)]          # 学習しない画像データ
        Y_TEST_list  += [target] * tsnum;                                     # 学習しない分類番号
        target += 1

    X_TRAIN = np.array(X_TRAIN_list + X_TEST_list)    # 連結
    Y_TRAIN = np.array(Y_TRAIN_list + Y_TEST_list)    # 連結
    print(">> 学習サンプル数 : ", X_TRAIN.shape)
    y_train = np_utils.to_categorical(Y_TRAIN, target)    # 自然数をベクトルに変換
    valrate = tsnum * target * 1.0 / X_TRAIN.shape[0]

    # 学習率の変更
    class Schedule(object):
        def __init__(self, init=0.001):      # 初期値定義
            self.init = init
        def __call__(self, epoch):           # 現在値計算
            lr = self.init
            for i in range(1, epoch+1):
                lr *= learn_schedule
            return lr

    def get_schedule_func(init):
        return Schedule(init)

    lrs = LearningRateScheduler(get_schedule_func(0.001))
    mcp = ModelCheckpoint(filepath='best.hdf5', monitor='val_loss', verbose=1, save_best_only=True, mode='auto')
    model = BuildCNN(ipshape=(X_TRAIN.shape[1], X_TRAIN.shape[2], X_TRAIN.shape[3]), num_classes=target)

    print(">> 学習開始")
    hist = model.fit(X_TRAIN, y_train,
                     batch_size=batch_size,
                     verbose=1,
                     epochs=nb_epoch,
                     validation_split=valrate,
                     callbacks=[lrs, mcp])

    json_string = model.to_json()
    json_string += '##########' + str(ClassNames)
    open('model.json', 'w').write(json_string)
    model.save_weights('last.hdf5')
    
    #acc, val_accのプロット
    plt.plot(hist.history["acc"], label="acc", ls="-", marker="o")
    plt.plot(hist.history["val_acc"], label="val_acc", ls="-", marker="x")
    plt.ylabel("accuracy")
    plt.xlabel("epoch")
    plt.legend(loc="best")
    plt.show()


################################
########## 試行・実験 ##########
################################
def TestProcess(imgname):
    modelname_text = open("model.json").read()
    json_strings = modelname_text.split('##########')
    textlist = json_strings[1].replace("[", "").replace("]", "").replace("\'", "").split()
    model = model_from_json(json_strings[0])
    model.load_weights("last.hdf5")  # best.hdf5 で損失最小のパラメータを使用
    img = load_img(imgname, target_size=(hw["height"], hw["width"]))    
    TEST = img_to_array(img) / 255
    
    pred = model.predict(np.array([TEST]), batch_size=1, verbose=0)
    
    img = imgname.split("/")
    picture = img[1]
    gomi = img[2]
    
    Path = './picture2/' + picture
    
    Class = textlist[np.argmax(pred)].replace(",", "")
    
    print(Path)
    
    #Classに応じて画像をそれぞれのフォルダーに分類
    if Class == "soeda":
        shutil.copyfile(Path, './../../../Google ドライブ/soeda/' + picture)
    elif Class == "kaoru":
        shutil.copyfile(Path, './../../../Google ドライブ/kaoru/' + picture)
    elif Class == "kasuya":
        shutil.copyfile(Path, './../../../Google ドライブ/kasuya/' + picture)
    elif Class == "reiya":
        shutil.copyfile(Path, './../../../Google ドライブ/reiya/' + picture)
    elif Class == "mattyan":
        shutil.copyfile(Path, './../../../Google ドライブ/mattyan/' + picture)
    elif Class == "gomi":
        shutil.copyfile(imgname, './gomi/' + gomi)
    else:
        print("該当無し")
    
    
    print(">> 計算結果↓\n" + str(pred))
    print(">> この画像（"+ imgname +"）は「" + Class + "」です。")
    
    return np.argmax(pred)

In [9]:
import os.path as op

i = 0
for filename in FileNames :
    # ディレクトリ名入力
    while True :
        dirname = input(">>「" + ClassNames[i] + "」の画像のあるディレクトリ ： ")
        if op.isdir(dirname) :
            break
        print(">> そのディレクトリは存在しません！")

    # 関数実行
    PreProcess(dirname, filename, var_amount=3)
    i += 1
    print("¥")
    #img/kaoruO
    #img/soedaO
    #img/reiyaO
    #img/mattyanO
    #img/kasuyaO

>>「gomi」の画像のあるディレクトリ ： img/gomi
>> img/gomiから6801個のファイル読み込み成功
¥
>>「kaoru」の画像のあるディレクトリ ： img/kaoru
>> img/kaoruから273個のファイル読み込み成功
¥
>>「soeda」の画像のあるディレクトリ ： img/soeda
>> img/soedaから249個のファイル読み込み成功
¥
>>「reiya」の画像のあるディレクトリ ： img/reiya
>> img/reiyaから224個のファイル読み込み成功
¥
>>「mattyan」の画像のあるディレクトリ ： img/mattyan
>> img/mattyanから377個のファイル読み込み成功
¥
>>「kasuya」の画像のあるディレクトリ ： img/kasuya
>> img/kasuyaから286個のファイル読み込み成功
¥


In [10]:
# 関数実行
Learning(tsnum=10, nb_epoch=50, batch_size=8, learn_schedule=0.9)

>> 学習サンプル数 :  (24630, 32, 32, 3)
>> 学習開始
Train on 24570 samples, validate on 60 samples
Epoch 1/50
24570/24570 [==============================] - 1200s 49ms/step - loss: 0.5503 - acc: 0.8496 - val_loss: 2.0604 - val_acc: 0.2667

Epoch 00001: val_loss improved from inf to 2.06039, saving model to best.hdf5
Epoch 2/50
24570/24570 [==============================] - 1188s 48ms/step - loss: 0.4174 - acc: 0.8786 - val_loss: 1.8656 - val_acc: 0.3667

Epoch 00002: val_loss improved from 2.06039 to 1.86555, saving model to best.hdf5
Epoch 3/50
24570/24570 [==============================] - 1187s 48ms/step - loss: 0.3604 - acc: 0.8926 - val_loss: 1.6343 - val_acc: 0.3667

Epoch 00003: val_loss improved from 1.86555 to 1.63426, saving model to best.hdf5
Epoch 4/50
24570/24570 [==============================] - 1188s 48ms/step - loss: 0.3280 - acc: 0.9038 - val_loss: 1.4429 - val_acc: 0.4667

Epoch 00004: val_loss improved from 1.63426 to 1.44288, saving model to best.hdf5
Epoch 5/50
24570/24570 [

NameError: name 'plt' is not defined

### 単発

In [ ]:
import os.path as op

while True:
    while True:
        imgname = input("\n>> 入力したい画像ファイル(「END」で終了) ： ")
        if op.isfile(imgname) or imgname == "END":
            break
        print(">> そのファイルは存在しません！")
    if imgname == "END":
        break

    # 関数実行
    TestProcess(imgname)
    
    #test/reiya/IMG_0995.jpg

### 連発

In [5]:
dirname = "out_picture2"#input("フォルダ名：")
files = glob.glob(dirname + "/*/*.jpg")
cn1 = 0; cn2 = 0;
for imgname in files :
    kind = TestProcess(imgname)
    if kind == 1:
        cn2 += 1
    cn1 += 1

    #print(files)
print("学習・非学習含め正答率は" + str(cn2*1.0/cn1) + "です。")

./picture2/DSC_0173.JPG
>> 計算結果↓
[[3.6555839e-08 5.0864380e-04 1.9256660e-05 4.5833708e-04 9.9847633e-01
  5.3739682e-04]]
>> この画像（out_picture2/DSC_0173.JPG/out_(DSC_0173.JPG)4.jpg）は「mattyan」です。
./picture2/DSC_0957.JPG
>> 計算結果↓
[[3.8141877e-07 9.9856049e-01 1.1702053e-04 3.4924084e-04 2.4702353e-04
  7.2589022e-04]]
>> この画像（out_picture2/DSC_0957.JPG/out_(DSC_0957.JPG)10.jpg）は「kaoru」です。
./picture2/DSC_0957.JPG
>> 計算結果↓
[[1.0000000e+00 4.5653308e-09 4.8208584e-08 1.1725707e-08 3.1915048e-10
  4.6425672e-10]]
>> この画像（out_picture2/DSC_0957.JPG/out_(DSC_0957.JPG)11.jpg）は「gomi」です。
./picture2/DSC_0957.JPG
>> 計算結果↓
[[3.0795280e-05 9.0187008e-05 9.9586970e-01 1.3028311e-04 3.5478105e-04
  3.5243130e-03]]
>> この画像（out_picture2/DSC_0957.JPG/out_(DSC_0957.JPG)21.jpg）は「soeda」です。
./picture2/DSC_0957.JPG
>> 計算結果↓
[[2.7079109e-04 1.5494004e-01 1.0229335e-02 3.8832930e-04 4.9004483e-04
  8.3368146e-01]]
>> この画像（out_picture2/DSC_0957.JPG/out_(DSC_0957.JPG)9.jpg）は「kasuya」です。
./picture2/IMG_0228.JPG
>> 計算結

./picture2/IMG_2514.JPG
>> 計算結果↓
[[1.0000000e+00 4.0758567e-11 6.8011485e-11 8.5785712e-11 2.9254069e-10
  8.4048150e-12]]
>> この画像（out_picture2/IMG_2514.JPG/out_(IMG_2514.JPG)1.jpg）は「gomi」です。
./picture2/IMG_2514.JPG
>> 計算結果↓
[[9.9997294e-01 8.6359669e-06 8.8173729e-06 3.5600968e-07 8.3512423e-06
  9.4743439e-07]]
>> この画像（out_picture2/IMG_2514.JPG/out_(IMG_2514.JPG)10.jpg）は「gomi」です。
./picture2/IMG_2514.JPG
>> 計算結果↓
[[9.9822098e-01 6.7871116e-04 6.4626039e-04 6.3289663e-05 3.0890471e-04
  8.1877602e-05]]
>> この画像（out_picture2/IMG_2514.JPG/out_(IMG_2514.JPG)11.jpg）は「gomi」です。
./picture2/IMG_2514.JPG
>> 計算結果↓
[[9.9999976e-01 7.5821468e-08 1.4165983e-08 1.6042529e-07 1.7987781e-09
  1.9254641e-10]]
>> この画像（out_picture2/IMG_2514.JPG/out_(IMG_2514.JPG)12.jpg）は「gomi」です。
./picture2/IMG_2514.JPG
>> 計算結果↓
[[1.0000000e+00 8.4917131e-09 1.4316960e-08 5.0837717e-10 2.2353978e-10
  1.6696818e-10]]
>> この画像（out_picture2/IMG_2514.JPG/out_(IMG_2514.JPG)13.jpg）は「gomi」です。
./picture2/IMG_2514.JPG
>> 計算結果↓
[[0

./picture2/thumb_IMG_2893_1024.jpg
>> 計算結果↓
[[9.9996614e-01 2.5586094e-06 2.4924662e-05 3.2704797e-08 5.3503272e-06
  1.0241130e-06]]
>> この画像（out_picture2/thumb_IMG_2893_1024.jpg/out_(thumb_IMG_2893_1024.jpg)5.jpg）は「gomi」です。
./picture2/thumb_IMG_2893_1024.jpg
>> 計算結果↓
[[9.9998081e-01 1.8090550e-06 7.1332311e-06 2.3647733e-06 6.0173725e-06
  1.9012944e-06]]
>> この画像（out_picture2/thumb_IMG_2893_1024.jpg/out_(thumb_IMG_2893_1024.jpg)6.jpg）は「gomi」です。
./picture2/thumb_IMG_2893_1024.jpg
>> 計算結果↓
[[9.9998522e-01 5.3466414e-07 3.0154038e-07 1.9448398e-06 1.1096648e-07
  1.1832605e-05]]
>> この画像（out_picture2/thumb_IMG_2893_1024.jpg/out_(thumb_IMG_2893_1024.jpg)7.jpg）は「gomi」です。
./picture2/thumb_IMG_2898_1024.jpg
>> 計算結果↓
[[4.2669321e-06 6.9294050e-02 1.2011196e-02 8.3833599e-01 7.9318330e-02
  1.0361187e-03]]
>> この画像（out_picture2/thumb_IMG_2898_1024.jpg/out_(thumb_IMG_2898_1024.jpg)1.jpg）は「reiya」です。
./picture2/thumb_IMG_2898_1024.jpg
>> 計算結果↓
[[1.00000000e+00 1.75064423e-12 1.26484045e-10 2.095637

./picture2/thumb_IMG_2906_1024.jpg
>> 計算結果↓
[[1.0000000e+00 4.4400053e-08 1.1693473e-09 2.5439910e-09 6.6018084e-09
  6.8577015e-09]]
>> この画像（out_picture2/thumb_IMG_2906_1024.jpg/out_(thumb_IMG_2906_1024.jpg)5.jpg）は「gomi」です。
./picture2/thumb_IMG_2912_1024.jpg
>> 計算結果↓
[[4.0482289e-09 1.7749829e-07 9.9366778e-01 8.8619583e-07 4.5302017e-03
  1.8009512e-03]]
>> この画像（out_picture2/thumb_IMG_2912_1024.jpg/out_(thumb_IMG_2912_1024.jpg)1.jpg）は「soeda」です。
./picture2/thumb_IMG_2914_1024.jpg
>> 計算結果↓
[[5.9482130e-07 2.3154096e-05 9.9116254e-01 1.8544143e-04 7.5725252e-03
  1.0557951e-03]]
>> この画像（out_picture2/thumb_IMG_2914_1024.jpg/out_(thumb_IMG_2914_1024.jpg)1.jpg）は「soeda」です。
./picture2/thumb_IMG_2915_1024.jpg
>> 計算結果↓
[[1.5837068e-07 2.2636943e-01 1.4423192e-04 6.1149331e-04 7.7285624e-01
  1.8385219e-05]]
>> この画像（out_picture2/thumb_IMG_2915_1024.jpg/out_(thumb_IMG_2915_1024.jpg)1.jpg）は「mattyan」です。
./picture2/thumb_IMG_2916_1024.jpg
>> 計算結果↓
[[0.00874978 0.7728705  0.14441273 0.01805441 0.019